# Feuille de travaux pratiques. Formules de quadrature

In [ ]:
# chargement des bibliothèques
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

## Exercice 1 (ordre de convergence de formules de quadrature composées)
Soit $f$ une fonction réelle continue sur un intervalle $[a,b]$. Le but de cet exercice est de mesurer l'efficacité de [formules de quadrature de Newton-Cotes](https://fr.wikipedia.org/wiki/Formule_de_Newton-Cotes) composées classiques pour l'approximation de l'intégrale définie
$$
I(f)=\int_a^b f(x)\,\mathrm{d}x,
$$
pour différents choix fonction. En particulier, on souhaite observer l'influence de la régularité de la fonction sur la précision des formules.

**1.** Écrire trois fonctions, ayant chacune pour paramètres d'entrée la fonction $f$, les bornes $a$ et $b$ de l'intervalle, et un nombre entier non nul $m$ de subdivisions de l'intervalle, calculant une approximation $I_{m,n}(f)$ de $I(f)$ respectivement par les formules de quadrature de la [règle du point milieu](http://fr.wikipedia.org/wiki/M%C3%A9thode_du_point_m%C3%A9dian) (formule ouverte, $n=0$), de la [règle du trapèze](http://fr.wikipedia.org/wiki/M%C3%A9thode_des_trap%C3%A8zes) (formule fermée, $n=1$) et de la [règle de Simpson](http://fr.wikipedia.org/wiki/M%C3%A9thode_de_Simpson) (formule fermée, $n=2$) composées.

In [ ]:
def point_milieu_composee(f,a,b,m):
    somme=0.
    for j in range(m):
        somme+=f(a+(b-a)*(j+0.5)/m)
    return (b-a)*somme/m

def trapeze_composee(f,a,b,m):
    somme=0.
    for j in range(m-1):
        somme+=2.*f(a+(j+1)*(b-a)/float(m))
    somme+=f(a)+f(b)
    return (b-a)*0.5*somme/m

def simpson_composee(f,a,b,m):
    somme=0.
    for j in range(m):
        somme+=f(a+j*(b-a)/float(m))+4.*f(a+(j+0.5)*(b-a)/m)+f(a+(j+1)*(b-a)/float(m))
    return (b-a)*somme/(6.*m)

**2.** Au moyen de la commande `semilogy`, tracer le graphe des courbes de l'erreur $\vert I(f)-I_{m,n}(f)\vert$ de chacune de ces trois formules en fonction du nombre de sous-intervalles pour $f(x)=e^x$, $a=0$ et $b=1$. Commenter les résultats.

In [ ]:
mmax=100
erreur=zeros((3,mmax))
I=exp(1)-1

for m in range(mmax):
    erreur[0,m]=abs(point_milieu_composee(exp,0,1,m+1)-I)
    erreur[1,m]=abs(trapeze_composee(exp,0,1,m+1)-I)
    erreur[2,m]=abs(simpson_composee(exp,0,1,m+1)-I)
    
semilogy(arange(1,mmax+1),erreur[0],label='erreur point milieu composée')
semilogy(arange(1,mmax+1),erreur[1],label='erreur trapèze composée composée')
semilogy(arange(1,mmax+1),erreur[2],label='erreur Simpson composée')
legend()

**3.** Reprendre la question précédente pour $f(x)=\vert 3\,x^4-1\vert$, $a=0$ et $b=1$. Que constate-t-on? Comment procéder pour retrouver les ordres de convergence théoriques des formules dans ce cas?

In [ ]:
def f(x):
    return abs(3.*x**4-1.)

alpha=3**(-0.25)
I=(4*alpha-1)*2/5

for m in range(mmax):
    erreur[0,m]=abs(point_milieu_composee(f,0,1,m+1)-I)
    erreur[1,m]=abs(trapeze_composee(f,0,1,m+1)-I)
    erreur[2,m]=abs(simpson_composee(f,0,1,m+1)-I)

semilogy(arange(1,mmax+1),erreur[0],label='erreur point milieu composée')
semilogy(arange(1,mmax+1),erreur[1],label='erreur trapèze composée composée')
semilogy(arange(1,mmax+1),erreur[2],label='erreur Simpson composée')
legend()




## Exercice bonus (méthode de Romberg)

On considère l'utilisation de la règle du trapèze composée associée à une subdivision dyadique de l'intervalle $[a,b]$ pour le calcul de l'intégrale $I(f)$ de
l'exercice précédent. En supposant la fonction $f$ suffisamment régulière et en posant $H=\frac{b-a}{2^k}$, avec $k$ un entier naturel.

On peut montrer à partir de la [formule d'Euler-Maclaurin](https://fr.wikipedia.org/wiki/Formule_d%27Euler-Maclaurin) que l'erreur vérifie
$$
I(f)-I_{2^k,1}(f)=c_1\,H^2+c_2\,H^4+\dots,
$$
où les coefficients $c_k$ ne dépendent pas de $H$ et où seules les puissances paires apparaissent dans le développement. On peut exploiter cette propriété pour supprimer un à un les termes du développement, et ainsi obtenir de meilleures approximations de l'intégrale $I(f)$.

Posons $R_{k,0}=I_{2^k,1}(f)$. À partir d'une estimation de l'intégrale pour une subdivision de taille $\frac{H}{2}$, notée $R_{k+1,0}(=I_{2^{k+1},1}(f))$, l'utilisation du [procédé d'extrapolation de Richardson](https://fr.wikipedia.org/wiki/Extrapolation_de_Richardson) permet d'éliminer le premier terme du développement en considérant l'approximation fournie par la quantité
$$
R_{k+1,1}=R_{k+1,0}+\frac{1}{3}(R_{k+1,0}-R_{k,0})=\frac{1}{3}(4\,R_{k+1,0}-R_{k,0}),
$$
conduisant ainsi à une erreur d'ordre quatre en $H$.

Le procédé appelé [méthode de Romberg](https://fr.wikipedia.org/wiki/M%C3%A9thode_de_Romberg) pour l'évaluation approchée de $I(f)$ consiste en l'application répétée de cette opération à partir de la valeur $k=0$.

**1.** Écrire une fonction `romberg`, ayant pour arguments d'entrée une fonction $f$, les bornes $a$ et $b$ de l'intervalle, un nombre d'extrapolations maximal $N$ et une tolérance $\varepsilon$, renvoyant la valeur de l'approximation $R_{k,k}$ telle que $|R_{k,k}-R_{k-1,k-1}|\leq\varepsilon$, avec $0\leq k\leq N$, ou bien $k=N$. Pour cela, on construira un tableau contenant les valeurs extrapolées $R_{k,m}$, $0\leq m\leq k\leq N$, dont les éléments satisfont la relation de récurrence
$$
R_{k,m}=R_{k,m-1}+\frac{1}{4^m-1}\left(R_{k,m-1}-R_{k-1,m-1}\right),\ 1\leq m\leq k\leq N.
$$
Les approximations successives de $R_{k,k}$ construites par la méthode de Romberg se trouvent alors sur la diagonale du tableau et les éléments de la première colonne du tableau sont tels que $R_{k,0}=I_{2^k,1}(f)$, $k=0,\dots,N$.

In [2]:
def romberg(f,a,b,itermax,tol):
# règle du trapèze
    H=b-a
    table=zeros((itermax+1,itermax+1))
    table[0,0]=H*(f(a)+f(b))/2.

    for k in range(1,itermax+1):
# règle du trapèze composée
        H=H/2.
        table[k,0]=0.5*table[k-1,0]
        somme=0.
        for i in range(1,2**k+1,2):
            somme=somme+f(a+i*H)
        table[k,0]=table[k,0]+H*somme
# extrapolation de Richardson
        for m in range(1,k+1):
            table[k,m]=table[k,m-1]+(table[k,m-1]-table[k-1,m-1])/(4**m-1.)
        if (abs(table[k,k]-table[k-1,k-1])<=tol):
            return table[k,k]
    print('Le nombre maximum d\'itérations a été atteint sans que le critêre d\'arrêt soit satisfait avec la tolérance voulue.')
    return table[k,k]

**2.** Utiliser cette fonction pour calculer des approximations de l'intégrale $I(f)$ dans les cas suivants :
* $f(x)=x^3$, $a=0$, $b=1$,
* $f(x)=\frac{1}{x}$, $a=1$, $b=2$,
* $f(x)=\sin(x)$, $a=0$, $b=\pi$,
* $f(x)=x\sin(\sqrt{x})$, $a=0$, $b=3$,
* $f(x)=e^{-x^2}$, $a=0$, $b=1$.

In [4]:
itermax=100
tol=1e-10
def f(x):
    return x**3

print('valeur de l\'intégrale : ',0.25)
print('approximation obtenue : ',romberg(f,0.,1.,itermax,tol))



valeur de l'intégrale :  0.25


NameError: name 'zeros' is not defined

In [5]:
def f(x):
    return 1./x

print('valeur de l\'intégrale : ',log(2))
print('approximation obtenue : ',romberg(f,1.,2.,itermax,tol))

print('valeur de l\'intégrale : ',2.)
print('approximation obtenue : ',romberg(sin,0.,pi,itermax,tol))

def f(x):
    return x*sin(sqrt(x))

print('valeur de l\'intégrale : ',6.*(sin(sqrt(3.))+sqrt(3)*cos(sqrt(3))))
print('approximation obtenue : ',romberg(f,0.,3.,itermax,tol))

def f(x):
    return exp(-x*x)

from scipy import special
print('valeur de l\'intégrale : ',0.5*sqrt(pi)*special.erf(1.))
print('approximation obtenue : ',romberg(f,0.,1.,itermax,tol))

NameError: name 'log' is not defined